In [1]:
import sqlite3
from sqlite3 import Error
from tkinter import Tk, filedialog
import pandas as pd
import numpy as np
import random

# file selection packages 
import traitlets
from IPython.display import display
from ipywidgets import widgets
from tkinter import Tk, filedialog

import ipywidgets as ipw
from ipywidgets import interact, interact_manual

from tkinter import *
import tkinter as tk


In [25]:
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self, *args, **kwargs):
        """Initialize the SelectFilesButton class."""
        super(SelectFilesButton, self).__init__(*args, **kwargs)
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.
        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True)

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"
file = SelectFilesButton()
file

SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

In [238]:
# Load in file as a DataFrame
new_df = pd.read_csv(file.files[0])

In [239]:
# drop the duplicate sentences
new_df.drop_duplicates(subset='Input.text', keep='first', inplace=True)

# mix the df by a random sequence of numbers
for i in range(5):
    random_list = random.sample(range(len(new_df['Input.text'])), len(new_df['Input.text']))
    new_df['scramble_seq'] = random_list # add the list to the dataframe
    new_df = new_df.sort_values(by='scramble_seq')
    

# drop the random number sequence column that scrambles
new_df.drop(columns=['scramble_seq'], inplace=True) # drop the column used to scramble data
new_df.reset_index(drop=True, inplace=True) # reset the index

In [242]:
# Ask for user input to see what they want to chnage
# IMPORTANT NOTE: If filtering by confidence value must type ALL decimal places with '%' (e.g. '100.00%')

for thing in new_df.columns:
    filt = input("Do you want to filter by " + str(thing) + " ? (y/n)")
    if filt == 'y':
        by = input('type what you want ' + str(thing) + ' to be')
        if thing == 'is_stressor' or thing == 'is_covid':
            by = int(by)
        new_df = new_df[new_df[thing] == by]
    more = input('Do you want to filter by another aspect? (y/n)')
    if more == 'n':
        break

 

Do you want to filter by sID ? (y/n)
Do you want to filter by another aspect? (y/n)
Do you want to filter by Input.text ? (y/n)
Do you want to filter by another aspect? (y/n)
Do you want to filter by is_stressor ? (y/n)y
type what you want is_stressor to be1
Do you want to filter by another aspect? (y/n)
Do you want to filter by is_stressor_conf ? (y/n)y
type what you want is_stressor_conf to be100.00%
Do you want to filter by another aspect? (y/n)n


In [243]:
# Creating the test set list

test_set = [] # create big list that has all the sentences that are going to make the test set across all labels
labels = new_df['top_label'].unique().tolist()
for label in labels:
    sentence_list = new_df[new_df['top_label'] == label]['Input.text'].tolist()
    count = len(sentence_list)
    test_amount = count * 0.2
    added = 0
    while added <= test_amount:
        index = random.randint(0, count - 1)
        test_set.append(sentence_list[index])       
        del sentence_list[index]
        added += 1
        count -= 1

In [244]:
# Create a list of 0's
set_list = []
sentences = new_df['Input.text'].tolist()
for i in range(len(new_df['top_label'])):
    set_list.append(0)

# put a 1 in the row where a testing sentence is 
for sentence in test_set:
    index = sentences.index(sentence)
    set_list[index] = 1

# add the testing and training list to the dataframe 
new_df['set'] = set_list

In [245]:
# create a dictionary that holds a value for each label
key_dict = {}
for i in range(len(labels)):
    key_dict[labels[i]] = i

# create a list that is going to represent each label as the value from the dictionary    
Class = []
for label in new_df['top_label']:
    Class.append(key_dict[label])

# add the column to the dataframe    
new_df['class'] = Class 

In [250]:
for label in labels:
    temp_list = [0 for x in range(len(new_df['Input.text']))]
    for i in range(len(new_df['top_label'])):
        top = new_df['top_label'].tolist()[i]
        if top == label:
                   temp_list[i] = 1
    new_df[str(label) + '_' + 'binary'] = temp_list

In [246]:
# key for the 'class' column
print(key_dict)

{'School': 0, 'Emotional Turmoil': 1, 'Work': 2, 'Social Relationships': 3, 'Health, Fatigue, or Physical Pain': 4, 'Financial Problem': 5, 'Everyday Decision Making': 6, 'Family Issues': 7, 'Other': 8}


In [247]:
new_df
# should I reset the index again here? 
# Is this what we want the form to look like?

,sID,Input.text,is_stressor,is_stressor_conf,is_covid,is_covid_conf,top_label,second_label,avg_severity,median_severity,...,Emotional Turmoil,School,Family Issues,Social Relationships,Work,"Health, Fatigue, or Physical Pain",Source,is_seed,set,class
0,5065,that makes my schoolwork harder to manage.,1,100.00%,0,100.00%,School,Other,2.6,2,...,0.0,0.9,0.0,0.0,0.0,0.0,Inquire,0,0,0
1,2711,The situation with coronavirus has been very s...,1,100.00%,1,100.00%,Emotional Turmoil,Everyday Decision Making,4.8,4,...,0.4,0.0,0.0,0.0,0.0,0.1,mTurk_synthetic_covid,0,0,1
2,97,Being laid off from work,1,100.00%,1,60.00%,Work,Family Issues,6.4,7,...,0.1,0.0,0.2,0.0,0.5,0.1,mTurk_synthetic_covid,0,0,2
3,5287,"You are dependent on your friends, but you are...",1,100.00%,0,100.00%,Social Relationships,Other,5.2,4,...,0.1,0.0,0.0,0.7,0.0,0.0,Inquire,0,0,3
4,2424,school shootings,1,100.00%,0,100.00%,School,Emotional Turmoil,6.4,7,...,0.3,0.6,0.0,0.1,0.0,0.0,mTurk_synthetic,0,0,0
6,1197,"i work 60 plus hour weeks, i'm stressed beyond...",1,100.00%,0,100.00%,Work,Emotional Turmoil,7.4,7,...,0.2,0.0,0.0,0.0,0.7,0.1,mTurk_synthetic,0,0,2
8,6666,he never seems to think that anything is his p...,1,100.00%,0,100.00%,Social Relationships,Other,2.8,3,...,0.1,0.0,0.2,0.4,0.0,0.0,Inquire,0,0,3
9,3361,I haven't been sleeping well lately for worry.,1,100.00%,0,100.00%,"Health, Fatigue, or Physical Pain",Other,3.8,4,...,0.1,0.0,0.0,0.0,0.0,0.8,Inquire,0,1,4
11,660,i am taking 18 credits this semester.,1,100.00%,0,100.00%,School,,5.6,5,...,0.0,1.0,0.0,0.0,0.0,0.0,mTurk_synthetic,1,0,0
12,6160,This stupid money situation is coming down on ...,1,100.00%,0,100.00%,Financial Problem,Everyday Decision Making,5.8,6,...,0.0,0.0,0.0,0.0,0.0,0.0,Inquire,0,0,5


In [3]:
test_df = pd.read_csv(file.files[0])

In [265]:
master = Tk()

def var_states():
   print("male: %d,\nfemale: %d" % (var1.get(), var2.get()))

Label(master, text="Your sex:").grid(row=0, sticky=W)
var1 = IntVar()
Checkbutton(master, text="male", variable=var1).grid(row=1, sticky=W)
var2 = IntVar()
Checkbutton(master, text="female", variable=var2).grid(row=2, sticky=W)
Button(master, text='Quit', command=master.quit).grid(row=3, sticky=W, pady=4)
Button(master, text='Show', command=var_states).grid(row=4, sticky=W, pady=4)
mainloop()
root.withdraw()

''

In [16]:
def show():
    myLabel = Label(root, text=var.get()).pack()

In [20]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
 
import tkinter as tk
 

window = tk.Tk()
window.title('My Window')
window.geometry('100x100')
 
l = tk.Label(window, bg='white', width=20, text='empty')
l.pack()
 
def print_selection():
    if (var1.get() == 1) & (var2.get() == 0):
        l.config(text='I love Python ')
    elif (var1.get() == 0) & (var2.get() == 1):
        l.config(text='I love C++')
    elif (var1.get() == 0) & (var2.get() == 0):
        l.config(text='I do not anything')
    else:
        l.config(text='I love both')

var1 = tk.IntVar()
var2 = tk.IntVar()
c1 = tk.Checkbutton(window, text='Python',variable=var1, onvalue=1, offvalue=0, command=print_selection)
c1.pack()
c2 = tk.Checkbutton(window, text='C++',variable=var2, onvalue=1, offvalue=0, command=print_selection)
c2.pack()
Button(root, text='Done', command= root.quit).pack()
 
window.mainloop()
window.withdraw()

KeyboardInterrupt: 

In [19]:
root = Tk()
root.geometry('400x400')
var = IntVar()
var2 = IntVar()
c = Checkbutton(root, text="test", variable=var)
c.pack()
Button(root, text='Done', command= root.quit).pack()
Checkbutton(root, text='test2', variable=var2).pack()
myButton = Button(root, text='get values', command=show).pack()

root.mainloop()
root.withdraw()

''

In [18]:
var.get()

0

In [ ]:
def is_stressor_drop_down():
    c = Checkbutton()

In [ ]:
var = Button(root, text='button', command=is_stressor_drop_down)

In [26]:
df = pd.read_csv(file.files[0])

In [29]:
 step = df[df['Source'] == 'Inquire']

In [31]:
print(len(step[step['Set'] == 0]['Set']))
print(len(step[step['Set'] == 1]['Set']))

1416
372


In [5]:
df

,sID,Input.text,is_stressor,is_stressor_conf,is_covid,is_covid_conf,top_label,second_label,avg_severity,median_severity,...,Other,Everyday Decision Making,Emotional Turmoil,School,Family Issues,Social Relationships,Work,"Health, Fatigue, or Physical Pain",Source,is_seed
0,0,?,0,100.00%,0,100.00%,Other,,0.0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,popbots_live,0
1,1,2 of my foster kittens died,1,100.00%,0,100.00%,Emotional Turmoil,Other,8.2,8,...,0.1,0.0,0.9,0.0,0.0,0.0,0.0,0.0,popbots_live,1
2,2,A better question would be what is not stressi...,1,100.00%,0,80.00%,Everyday Decision Making,Other,5.4,5,...,0.3,0.6,0.1,0.0,0.0,0.0,0.0,0.0,mTurk_synthetic_covid,0
3,3,a big project at work.,0,60.00%,0,100.00%,Other,Work,2.2,0,...,0.6,0.0,0.0,0.0,0.0,0.0,0.4,0.0,mTurk_synthetic,0
4,4,a certain person at work is being really angry...,1,100.00%,0,100.00%,Work,Social Relationships,3.4,3,...,0.0,0.0,0.0,0.0,0.0,0.3,0.7,0.0,mTurk_synthetic,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6892,6892,"I will not be attending school tomorrow, im no...",1,100.00%,0,100.00%,School,Emotional Turmoil,3.6,4,...,0.0,0.0,0.1,0.9,0.0,0.0,0.0,0.0,Inquire,0
6893,6893,"It's not fair to me, her, or my husband and ch...",1,100.00%,0,100.00%,Family Issues,Social Relationships,4.8,4,...,0.1,0.0,0.1,0.0,0.6,0.2,0.0,0.0,Inquire,0
6894,6894,God knows my mom and dad don't want me.,1,100.00%,0,100.00%,Family Issues,Emotional Turmoil,6.0,6,...,0.0,0.0,0.2,0.0,0.7,0.1,0.0,0.0,Inquire,0
6895,6895,Got into another fight with my brother.,1,100.00%,0,100.00%,Family Issues,Social Relationships,3.6,4,...,0.0,0.0,0.1,0.0,0.8,0.1,0.0,0.0,Inquire,0


In [18]:
new_df = df[df['Source'] == 'Inquire']

In [19]:
other = new_df[new_df['top_label'] == 'Emotional Turmoil']

In [20]:
nexte = other[other['is_stressor'] == 1]

In [21]:
al = nexte[nexte['is_stressor_conf'] == '100.00%']

In [23]:
fin = al[al['is_covid'] == 0]

In [24]:
fin[fin['is_covid_conf'] == '100.00%']

,sID,Input.text,is_stressor,is_stressor_conf,is_covid,is_covid_conf,top_label,second_label,avg_severity,median_severity,...,Other,Everyday Decision Making,Emotional Turmoil,School,Family Issues,Social Relationships,Work,"Health, Fatigue, or Physical Pain",Source,is_seed
3223,3223,"i'm tired. tired mentally, emotionally and phy...",1,100.00%,0,100.00%,Emotional Turmoil,"Health, Fatigue, or Physical Pain",5.4,5,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,Inquire,0
3288,3288,I'm not having a real good day today. I feel d...,1,100.00%,0,100.00%,Emotional Turmoil,"Health, Fatigue, or Physical Pain",5.6,5,...,0.1,0.0,0.5,0.0,0.0,0.0,0.0,0.4,Inquire,0
3301,3301,"So, I'm stressed out and I don't really know w...",1,100.00%,0,100.00%,Emotional Turmoil,Other,4.4,4,...,0.2,0.0,0.7,0.0,0.0,0.1,0.0,0.0,Inquire,0
3303,3303,"Days where I feel weak, or where I feel like s...",1,100.00%,0,100.00%,Emotional Turmoil,"Health, Fatigue, or Physical Pain",5.2,5,...,0.0,0.1,0.4,0.0,0.1,0.0,0.0,0.3,Inquire,0
3304,3304,i'm just not sure if i can handle it all becau...,1,100.00%,0,100.00%,Emotional Turmoil,Other,4.6,5,...,0.2,0.1,0.6,0.0,0.0,0.0,0.0,0.1,Inquire,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6785,6785,The feeling is so stressful.,1,100.00%,0,100.00%,Emotional Turmoil,Other,3.6,3,...,0.2,0.0,0.8,0.0,0.0,0.0,0.0,0.0,Inquire,0
6787,6787,i am having some inner turmoil reguarding my a...,1,100.00%,0,100.00%,Emotional Turmoil,Other,3.6,3,...,0.2,0.2,0.6,0.0,0.0,0.0,0.0,0.0,Inquire,0
6788,6788,"A lot of shit has happened, but I don't think ...",1,100.00%,0,100.00%,Emotional Turmoil,Other,4.2,5,...,0.4,0.0,0.6,0.0,0.0,0.0,0.0,0.0,Inquire,0
6790,6790,has damaged so many levels of myself.,1,100.00%,0,100.00%,Emotional Turmoil,Social Relationships,6.0,6,...,0.0,0.0,0.9,0.0,0.0,0.1,0.0,0.0,Inquire,0
